
# Bokeh for Time Series Analysis
<hr style="border: 2px solid black;">


<img src="./images/bokeh.png" alt="bokeh Logo" width="1000"/>
<hr style="border: 2px solid black;">

<img src="./images/bokeh_at_ag_glance.png" alt="bokeh Logo" width="1000"/>
<hr style="border: 2px solid black;">
**Introduction to Bokeh**
Bokeh is an interactive visualization library for Python that targets modern web browsers for presentation.
Unlike Matplotlib, which is primarily designed for static plots, Bokeh excels at creating
interactive plots and dashboards. It can handle large datasets and streaming data,
making it suitable for real-time applications.

**Key Features of Bokeh:**

* **Interactivity:** Built-in support for zooming, panning, hovering, and other interactive tools.
* **Web-Focused:** Generates HTML and JavaScript, making it easy to embed plots in web pages.
* **High Performance:** Can handle large datasets efficiently.
* **Versatility:** Supports a wide range of plot types (lines, bars, scatter plots, etc.).

<hr style="border: 2px solid black;">


**Documentation:**

For comprehensive documentation, please refer to the official Bokeh website: [https://docs.bokeh.org/en/latest/](https://docs.bokeh.org/en/latest/)


<hr style="border: 2px solid black;">


**Lab Exercise:**

Your task is to recreate the time series analysis lab we previously conducted using Pandas,
Matplotlib, and Seaborn, but this time, utilize the Bokeh library for visualization.
This will involve:

1.  Loading and preprocessing the "AirPassengersDates.csv" dataset.
2.  Creating interactive Bokeh plots for:
    * Time series line plots
    * Bar plots of aggregated data
    * Visualizing mean and standard deviation
    * Outlier detection
    * Resampling (upsampling and downsampling)
    * Lag analysis
    * Autocorrelation

Pay close attention to Bokeh's features for interactivity (tools, hover effects) and
its handling of data sources. Aim to replicate the insights and visualizations
from the previous lab while leveraging Bokeh's strengths.

Good luck!
<hr style="border: 2px solid black;">

In [1]:
!pip install bokeh



In [5]:
import pandas as pd

df = pd.read_csv('./datasets/AirPassengersDates.csv')
print(df.columns)


Index(['Date', '#Passengers'], dtype='object')


In [6]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import column, row
from bokeh.transform import dodge
from bokeh.palettes import Category10
from bokeh.models.tools import BoxZoomTool, ResetTool, PanTool, WheelZoomTool
import warnings
warnings.filterwarnings('ignore')

output_notebook()

# Chargement des données
df = pd.read_csv('./datasets/AirPassengersDates.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df.sort_index(inplace=True)
df.head()


Loading BokehJS ...

,#Passengers
Date,
1949-01-12,112
1949-02-24,118
1949-03-22,132
1949-04-05,129
1949-05-24,121


In [8]:
from bokeh.models import DatetimeTickFormatter

source = ColumnDataSource(data={
    'x': df.index,
    'y': df['#Passengers']
})

# Création de la figure sans formatters
p = figure(x_axis_type='datetime', title='Monthly Air Passengers Over Time',
           width=800, height=300, tools="pan,box_zoom,reset,wheel_zoom")

# Ajout du HoverTool avec le formateur de date
hover = HoverTool(
    tooltips=[
        ('Date', '@x{%F}'),
        ('Passengers', '@y')
    ],
    formatters={
        '@x': 'datetime'
    },
    mode='vline'
)

p.add_tools(hover)

# Tracés
p.line('x', 'y', source=source, line_width=2, color='navy', legend_label='Passengers')
p.circle('x', 'y', source=source, size=4, color='orange')

# Légendes et axes
p.legend.location = "top_left"
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Number of Passengers'

show(p)



In [10]:
# Agrégation annuelle
df_yearly = df.resample('Y').sum()
df_yearly.index = df_yearly.index.year.astype(str)  # Années en chaîne de caractères

# Création de la source Bokeh
source_year = ColumnDataSource(data=dict(
    year=list(df_yearly.index),  # On convertit explicitement en liste Python
    passengers=df_yearly['#Passengers']
))

# Création du graphique
p2 = figure(x_range=list(df_yearly.index), title="Total Passengers per Year",
            height=300, width=800, tools="hover",
            tooltips=[("Year", "@year"), ("Passengers", "@passengers")])

# Barres
p2.vbar(x='year', top='passengers', width=0.6, source=source_year, color=Category10[10][0])

# Axes
p2.xaxis.axis_label = 'Year'
p2.yaxis.axis_label = 'Passengers'
p2.xgrid.grid_line_color = None

show(p2)



In [11]:
rolling_mean = df['#Passengers'].rolling(window=12).mean()
rolling_std = df['#Passengers'].rolling(window=12).std()

source_stats = ColumnDataSource(data={
    'x': df.index,
    'y': df['#Passengers'],
    'mean': rolling_mean,
    'std_up': rolling_mean + rolling_std,
    'std_down': rolling_mean - rolling_std
})

p3 = figure(x_axis_type='datetime', title='Mean and Standard Deviation',
            width=800, height=300, tools="hover,pan,box_zoom,reset")

p3.line('x', 'y', source=source_stats, legend_label='Passengers', line_color='gray')
p3.line('x', 'mean', source=source_stats, legend_label='Mean', line_color='green')
p3.line('x', 'std_up', source=source_stats, legend_label='+1 Std Dev', line_color='blue', line_dash='dashed')
p3.line('x', 'std_down', source=source_stats, legend_label='-1 Std Dev', line_color='red', line_dash='dashed')

p3.legend.location = 'top_left'

show(p3)
